In [1]:
from io_processing.marpower import read_excel
import io_processing.sql.pika as io_pika
from pypika import Query, Column

%load_ext autoreload
%autoreload 2

In [2]:
df = read_excel("../io_lists/52422003_3210_AMCS IO-List R1.11.xlsx")
df

Rev.,Deleted,Target Type,Device,Module,Module Type,Terminal,Is Subscribe,Prefix Device,Tag,Cabinet,System,Yard Tag,Description,Parent,Redundant tag,P&ID,Cable ID,Cable Type,Cable Core No.,OPC UA Node ID,OPC UA Publish Interval,Modbus Slave Address,Modbus Data Type,Modbus Address,Modbus Source Type,Modbus Byte Size,Modbus Swap Words,Modbus Lower,Modbus Upper,Range Lower,Range Upper,Unit,Precision,Alert Code,Alert HH,Alert H,Alert L,Alert LL,Alert F,Delay On,Acknowledge Location,Sounding Locations,Alert priority,Intended Operator Response,Category A,Group Alarm,Call GEA on Alert,Disallow Inhibit,Vdr ID,General Lock Tag,General Lock Operator,General Lock Level,HH Lock Tag,HH Lock Operator,HH Lock Level,H Lock Tag,H Lock Operator,H Lock Level,L Lock Tag,L Lock Operator,L Lock Level,LL Lock Tag,LL Lock Operator,LL Lock Level,Do Not Log,Log To Daily Report,Log to CDP,Workstation,Timestamp
str,str,str,str,str,str,str,null,str,str,str,str,str,str,null,null,null,str,str,null,null,null,str,str,str,null,null,null,null,null,str,str,str,null,str,str,str,str,null,str,str,str,null,str,str,null,null,null,null,str,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,"""True""",null,null,null,null,null,null,null,null,null,null,null,"""ALARM CABINET - AMCS Main tech…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""R1.8""",null,"""Bool""","""KEB1""","""DI00""","""750-1415""","""1""",null,null,"""AmcsPowerSupplyFailure""","""DSK_SWR""","""4500 AMCS""",null,"""MTS AMCS 1 Power Supply Failur…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""4""",null,null,null,null,"""False""",null,"""TS""",null,"""Alarm""","""Check Primary (G2) supply, Sec…",null,null,null,null,"""1""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""R1.9""",null,"""Bool""","""KEB1""","""DI00""","""750-1415""","""2""",null,null,"""Spare_DI00_2""","""DSK_SWR""","""4500 AMCS""",null,"""Spare""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""5""",null,null,null,null,"""False""",null,"""TS""",null,"""Alarm""","""Check for earth fault""",null,null,null,null,"""1""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""R0.0""",null,"""Bool""","""KEB1""","""DI00""","""750-1415""","""3""",null,null,"""UmsEntranceUnitMannedSwitch""","""DSK_SWR""","""4500 AMCS""",null,"""UMS Entrance Unit Manned Switc…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""R0.0""",null,"""Bool""","""KEB1""","""DI00""","""750-1415""","""4""",null,null,"""UmsEntranceUnitUnmannedSwitch""","""DSK_SWR""","""4500 AMCS""",null,"""UMS Entrance Unit Unmanned Swi…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""R0.1 ""","""True""",null,"""KEC3_P4""",null,null,null,null,null,"""Serial__Var47""","""+CB.2""","""SPARE""",null,"""TODO""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,nul

In [3]:
root_topic = "amcs/+"


def mqtt_source(topic):
    mqtt_connector = io_pika.MqttConnector(
        "mqtt://vernemq:1883",
        topic,
        "at_least_once",
        "power-hub",
        "power-hub",
        "bytes",
    )

    table = Query.create_table("amcs").columns(
        io_pika.proctime_col(), io_pika.data_col()
    )

    return io_pika.table_with_connector(
        table, mqtt_connector, [io_pika.Include("partition", "topic")]
    )


print(str(mqtt_source(root_topic)))

CREATE TABLE "amcs" ("timestamp" TIMESTAMPTZ AS PROCTIME(),"data" BYTEA)
INCLUDE partition AS topic
WITH (connector = 'mqtt',
url = 'mqtt://vernemq:1883',
topic = 'amcs/+',
qos = 'at_least_once',
username = 'power-hub',
password = 'power-hub',
max_packet_size = 2000000
) FORMAT PLAIN ENCODE BYTES


In [7]:
import polars
import polars as pl
import yaml

from io_processing.components import Components
from io_processing.io_list import normalize_io_list, IoFlavor
from io_processing.sql.dbt import sql_for_io

io_list = normalize_io_list(df, IoFlavor.AMCS)
components = Components("../components.yaml")
orphans = components.orphan_signals(io_list)

a = io_list.select(
    "yard_tag", polars.struct(polars.exclude("yard_tag")).alias("signals")
)
grouped = (
    io_list.select(
        "yard_tag", polars.struct(polars.exclude("yard_tag")).alias("signals")
    ).group_by("yard_tag").all()
)

combined = components.combine_io(io_list, IoFlavor.AMCS).filter(pl.col("type") == "Pump")

a, b,c= sql_for_io(components._description, combined, IoFlavor.AMCS)
# a = (components.combine_io(io_list, IoFlavor.AMCS).with_columns(pl.col("signals").list.eval(pl.element().struct.field("value_name")).alias("value_names"))
#         .select(
#             pl.col("system"),
#             pl.col("type"),
#             pl.struct(pl.exclude("system", "type")).alias("components"),
#             pl.col("value_names")
#         )
#         .group_by([pl.col("system"), pl.col("type")])
#         .agg(pl.col("components"), pl.col("value_names").flatten()))
# missing = components.missing_components(io_list)

# io_list.join(components._components, ("yard_tag", "tag"), "left")
# a = components.combine_io(io_list).filter(pl.col("signals").list.len() > 1).with_columns(pl.col("signals").list.eval(pl.element().struct.field("Tag")).alias("tags")).select(pl.exclude("signals"))
# grouped
# io_list["type"].unique()

In [8]:
print(a)
print(b)

CREATE SOURCE "amcs" ("data" BYTEA,"timestamp" TIMESTAMPTZ AS PROCTIME())
INCLUDE partition AS topic
WITH (connector = 'mqtt',
url = 'mqtt://vernemq:1883',
topic = 'amcs/+',
qos = 'at_least_once',
username = 'power-hub',
password = 'power-hub',
max_packet_size = 2000000
) FORMAT PLAIN ENCODE BYTES
CREATE TABLE "signals" ("topic" VARCHAR,"yard_tag" VARCHAR,"value_name" VARCHAR,PRIMARY KEY ("topic"))


In [9]:
print(c["bilge_pump"][0])

INSERT INTO "signals" VALUES ('amcs/GEN_SERV_PUMP1_REM_AVAIL','21001001','remote_available'),('amcs/GEN_SERV_PUMP1_REM_START','21001001','remote_start'),('amcs/GEN_SERV_PUMP2_REM_AVAIL','21001002','remote_available'),('amcs/GEN_SERV_PUMP2_FIELD_START','21001002','field_start'),('amcs/GEN_SERV_PUMP2_FIELD_STOP','21001002','field_stop'),('amcs/GEN_SERV_PUMP2_REM_START','21001002','remote_start'),('amcs/BCKUP_GEN_SERV_PUMP_REM_AVAIL','21001003','remote_available'),('amcs/BCKUP_GEN_SERV_PUMP_FIELD_START','21001003','field_start'),('amcs/BCKUP_GEN_SERV_PUMP_FIELD_STOP','21001003','field_stop'),('amcs/BCKUP_GEN_SERV_PUMP_START_RUN','21001003','start_or_running'),('amcs/BCKUP_GEN_SERV_PUMP_FIELD_RUNN_FDBCK','21001003','field_feedback_running'),('amcs/STRIP_PUMP_REM_AVAIL','21001004','remote_available'),('amcs/STRIP_PUMP_RUNNING','21001004','running'),('amcs/GEN_SERV_PUMP2_FIELD_RUNN_FDBCK','21001004','field_feedback_running'),('amcs/STRIP_PUMP_START','21001004','start'),('amcs/STRIP_PUMP_STOP

In [10]:
print(c["bilge_pump"][1])

CREATE MATERIALIZED VIEW bilge_pumps AS WITH pivotted_signals AS (SELECT amcs.timestamp,signals.yard_tag,CASE WHEN signals.value_name='remote_start' THEN CAST(convert_from(amcs.data,'utf-8') AS BOOLEAN) ELSE NULL END remote_start,CASE WHEN signals.value_name='remote_available' THEN CAST(convert_from(amcs.data,'utf-8') AS BOOLEAN) ELSE NULL END remote_available,CASE WHEN signals.value_name='running' THEN CAST(convert_from(amcs.data,'utf-8') AS BOOLEAN) ELSE NULL END running,CASE WHEN signals.value_name='field_feedback_running' THEN CAST(convert_from(amcs.data,'utf-8') AS BOOLEAN) ELSE NULL END field_feedback_running,CASE WHEN signals.value_name='field_start' THEN CAST(convert_from(amcs.data,'utf-8') AS BOOLEAN) ELSE NULL END field_start,CASE WHEN signals.value_name='field_stop' THEN CAST(convert_from(amcs.data,'utf-8') AS BOOLEAN) ELSE NULL END field_stop,CASE WHEN signals.value_name='level_high' THEN CAST(convert_from(amcs.data,'utf-8') AS BOOLEAN) ELSE NULL END level_high,CASE WHEN si

In [20]:
from io_processing.data_gen.gen import Generator
import polars as pl

genny = Generator(combined, components._description, IoFlavor.AMCS)

In [199]:
import polars as pl

vitters_io = polars.read_excel("io_lists/hydraulic-plc/9428050-4 Sail-Hydraulic PLC IO list.xlsx", sheet_name="IO list", read_options={"header_row": 24})
vitters_cabling = polars.read_excel("io_lists/hydraulic-plc/9428051-4 Sail PLC cable list.xlsx", read_options={"header_row": 10})
with_components = vitters_io.with_columns(
  polars.col("PLC var-Name").str.extract(r"^([a-zA-Z_\d]+)\..*").alias("component")
)
joined_with_cabling = (
    with_components.filter((pl.col("component").is_not_null()))
    .join(vitters_cabling, left_on="1st cable no.", right_on="Cable no.", how="left")
    .select(
        pl.col("component"),
        pl.col("1st cable no."),
        pl.col("Source pos").alias("first source pos"),
        pl.col("Target pos").alias("first target"),
        pl.col("PLC var-Name"),
        pl.col("2nd cable no."),
    )
    .join(vitters_cabling, left_on="2nd cable no.", right_on="Cable no.", how="left")
    .select(
        pl.col("component"),
        pl.col("1st cable no."),
        pl.col("first source pos"),
        pl.col("first target"),
        pl.col("Source pos").alias("second source poc"),
        pl.col("Target pos").alias("second target"),
        pl.col("PLC var-Name"),
        pl.col("2nd cable no."),
    )
)

In [ ]:

first_cable_join = vitters_cabling.join(
  vitters_io,
  left_on="Cable no.",
  right_on="1st cable no."
)
second_cable_join = vitters_cabling.join(
  vitters_io,
  left_on="Cable no.",
  right_on="2nd cable no."
)

missing = (
    set(vitters_io["PLC var-Name"].to_list())
    - set(first_cable_join["PLC var-Name"].to_list())
    - set(second_cable_join["PLC var-Name"].to_list())
)

blabla = vitters_io.filter(polars.col("PLC var-Name").is_in(missing) & polars.col("IO box / Location").is_in(["KEEL", "MST"]))


# (polars.col("Cable no.") == polars.col("2nd cable no."))